In [1]:
import datetime, dateutil
import video_transect, misc
import ffmpeg
import pathlib
import pytz

In [2]:
# Paths
transect_dirs = ['/media/mha114/Massimal/Vega_Sola/Transects/Otter/Sola_02/20220820_160305_Sola02']


In [3]:
for tdir in transect_dirs:
    transect_dir = pathlib.Path(tdir)
    video_dir = transect_dir / 'GoPro'
    image_dir = transect_dir / 'Images'
    image_dir.mkdir(exist_ok=True)
    gpkg_dir = transect_dir / 'Images Geopackage'
    gpkg_dir.mkdir(exist_ok=True)
    gpkg_file = gpkg_dir / (transect_dir.stem + '_GeotaggedImages.gpkg')
    csv_file = misc.file_pattern_search(str(transect_dir),'*Gnss*.csv', recursive=True)[0]

    video_files = misc.file_pattern_search(str(video_dir), '*.[Mm][Pp]4')
    video_data = video_transect.get_video_data(str(video_dir))
    
    gdf = video_transect.otter_csv_to_geodataframe(csv_file)

In [4]:
otter_time_row = 48332
otter_key_time = gdf.Time.loc[otter_time_row]
print(f'Otter time (UTC) at key point is {otter_key_time}')

Otter time (UTC) at key point is 2022-08-20 17:23:38.884000+00:00


In [5]:
video_number = 7
video_ind = video_number - 1
video_start_time = dateutil.parser.parse(ffmpeg.probe(video_files[video_ind])['streams'][0]['tags']['creation_time'])
video_start_time = video_start_time.replace(tzinfo=None)  # Make naive
timezone = pytz.timezone('Europe/Oslo')
video_start_time = timezone.localize(video_start_time)
video_rel_time = datetime.timedelta(minutes=1,seconds=57)
video_key_time = video_start_time + video_rel_time
print(f'Video time (UTC+2) at key point is {video_key_time}')

Video time (UTC+2) at key point is 2022-08-20 18:54:27+02:00


In [6]:
video_offset = video_key_time - otter_key_time
print(f'Video time offset, taking timezones into account, (otter UTC+0, video UTC+2) is {video_offset}')
print(f'In seconds, the offset is {video_offset.total_seconds()} s')

Video time offset, taking timezones into account, (otter UTC+0, video UTC+2) is -1 days +23:30:48.116000
In seconds, the offset is -1751.884 s
